In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
ift_data = pd.read_excel('data/ift_data.xlsx')

In [3]:
ift_data.head()

,Gas,Water_content,time_minutes,volume_ratio,IFT
0,CH4,0.0,0.000050,1.000000,25.08
1,CH4,0.0,14.998333,1.002618,25.12
2,CH4,0.0,30.000000,1.005236,25.16
3,CH4,0.0,45.000000,1.006108,25.17
4,CH4,0.0,60.000000,1.007853,25.21


In [4]:
ift_data.info

<bound method DataFrame.info of      Gas  Water_content  time_minutes  volume_ratio    IFT
0    CH4            0.0      0.000050      1.000000  25.08
1    CH4            0.0     14.998333      1.002618  25.12
2    CH4            0.0     30.000000      1.005236  25.16
3    CH4            0.0     45.000000      1.006108  25.17
4    CH4            0.0     60.000000      1.007853  25.21
..   ...            ...           ...           ...    ...
556  CO2            0.7    150.000000      0.978155  19.93
557  CO2            0.7    165.000000      0.979369  19.95
558  CO2            0.7    180.000000      0.974515  19.87
559  CO2            0.7    195.000000      0.976942  19.92
560  CO2            0.7    210.000000      0.973301  19.86

[561 rows x 5 columns]>

In [5]:
#data = 


In [50]:
data = ift_data.iloc[:,:3]
y = ift_data.iloc[:,4]
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=123, shuffle=True)
numeric_features = ['Water_content', 'time_minutes']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Gas']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
reg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('Regressor', Ridge())])
param_grid = [{
    'Regressor__alpha':[0.1,0.25,0.4],
}]
search = GridSearchCV(reg, param_grid, n_jobs=-1)

search.fit(X_train, y_train)
search.best_params_
print("model score: %.3f" % search.score(X_test, y_test))
#reg.get_params().keys()

model score: 0.515


In [55]:
model_intercept = search.best_estimator_['Regressor'].intercept_
model_intercept

22.6419145110395

In [54]:
model_slopes = search.best_estimator_['Regressor'].coef_
model_slopes


array([-0.28426813, -0.15528172,  1.68815898, -1.68815898])

In [56]:
coeff_parameter = pd.DataFrame(search.best_estimator_['Regressor'].coef_,columns=['Coefficient'])
coeff_parameter

,Coefficient
0,-0.284268
1,-0.155282
2,1.688159
3,-1.688159


In [57]:
y_pred = search.predict(X_test)

In [58]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 1.0916569015462978
Mean Squared Error: 2.092540826080975
Root Mean Squared Error: 1.4465617256380645


### Model Selection : Trying Gradient boosting

In [59]:
data = ift_data.iloc[:,:3]
y = ift_data.iloc[:,4]
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=13, shuffle=True)
numeric_features = ['Water_content', 'time_minutes']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Gas']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
reg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('Regressor', GradientBoostingRegressor())])
param_grid = [{
    'Regressor__learning_rate':[0.1,0.25,0.4],
}]
search = GridSearchCV(reg, param_grid, n_jobs=-1)

search.fit(X_train, y_train)
search.best_params_
print("model score: %.3f" % search.score(X_test, y_test))
#reg.get_params().keys()

model score: 0.997


In [60]:
y_pred = search.predict(X_test)

In [61]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


Mean Absolute Error: 0.09117596089378016
Mean Squared Error: 0.014899983651306917
Root Mean Squared Error: 0.12206548919046249


In [70]:
X_test

,Gas,Water_content,time_minutes
469,CO2,0.10,240.0
168,CH4,0.20,720.0
39,CH4,0.00,600.0
98,CH4,0.10,615.0
198,CH4,0.33,450.0
...,...,...,...
395,CH4,0.70,930.0
35,CH4,0.00,540.0
380,CH4,0.70,705.0
187,CH4,0.33,120.0


In [87]:
y_test

469    19.00
168    24.46
39     24.40
98     23.42
198    23.86
       ...  
395    25.26
35     24.55
380    25.26
187    24.36
235    23.11
Name: IFT, Length: 113, dtype: float64

In [113]:
new_data = X_test.iloc[[0]]
new_data

,Gas,Water_content,time_minutes
469,CO2,0.1,240.0


In [86]:
search.predict(new_data)

array([19.03893469])

In [121]:
new_data.iloc[0,1] = 0.7

/Users/mosal/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [122]:
new_data


,Gas,Water_content,time_minutes
469,CO2,0.7,240.0


In [123]:
search.predict(new_data)

array([19.72457656])